In [30]:
import boto3

## Reset every staging tier except for Landing

In [ ]:
!aws s3 rm s3://test-lf-wm/etl/raw --recursive 
!aws s3 rm s3://test-lf-wm/etl/access --recursive 
!aws s3 rm s3://test-lf-wm/etl/optimised --recursive 

## Load source data into `Bucket.TEST` and `Bucket.PROD` if needed

In [4]:
!aws s3 ls s3://project-lf/etl/landing --recursive

2023-06-03 00:21:47      22655 etl/landing/claim_db/claim/full/202305211851-claim-full.csv
2023-06-03 00:21:51       5818 etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv
2023-06-03 00:21:55        814 etl/landing/claim_db/provider/full/202305221136-provider-full.csv


In [1]:
!aws s3 cp /home/glue_user/project_lf/ETL-TDD/mock/source_files/202305211851-claim-full.csv s3://test-lf-ap/etl/landing/claim_db/claim/full/202305211851-claim-full.csv

!aws s3 cp /home/glue_user/project_lf/ETL-TDD/mock/source_files/202305221132-policyholder-full.csv s3://test-lf-ap/etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv

!aws s3 cp /home/glue_user/project_lf/ETL-TDD/mock/source_files/202305221136-provider-full.csv s3://test-lf-ap/etl/landing/claim_db/provider/full/202305221136-provider-full.csv

upload: ../mock/source_files/202305211851-claim-full.csv to s3://test-lf-ap/etl/landing/claim_db/claim/full/202305211851-claim-full.csv
upload: ../mock/source_files/202305221132-policyholder-full.csv to s3://test-lf-ap/etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv
upload: ../mock/source_files/202305221136-provider-full.csv to s3://test-lf-ap/etl/landing/claim_db/provider/full/202305221136-provider-full.csv


In [88]:
env = Bucket.TEST

files = {
    "/home/glue_user/project_lf/ETL-TDD/mock/source_files/202305211851-claim-full.csv": f"s3://{env}/etl/landing/claim_db/claim/full/202305211851-claim-full.csv",
    "/home/glue_user/project_lf/ETL-TDD/mock/source_files/202305221132-policyholder-full.csv": f"s3://{env}/etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv",
    "/home/glue_user/project_lf/ETL-TDD/mock/source_files/202305221136-provider-full.csv": f"s3://{env}/etl/landing/claim_db/provider/full/202305221136-provider-full.csv",
}

# s3_resource = S3Resource.getInstance()

for src, dest in files.items():

    print(bucket_name, key)

test-lf-wm etl/landing/claim_db/claim/full/202305211851-claim-full.csv
test-lf-wm etl/landing/claim_db/policyholder/full/202305221132-policyholder-full.csv
test-lf-wm etl/landing/claim_db/provider/full/202305221136-provider-full.csv


## Delete every glue job

In [23]:
# delete all glue ETL jobs
glue = boto3.client('glue', region_name='ap-southeast-2')
jobs = glue.get_jobs()
for job in jobs['Jobs']:
    if job['Name'].startswith('stage_'):
        glue.delete_job(JobName=job['Name'])

## Refresh every .py job file in `code/jobs/` and `code/dependencies` 

In [50]:
# delete the libraries.zip file at s3://test-project-wm/code/libraries.zip
!aws s3 rm s3://test-project-wm/code/libraries.zip

# write cli that copies libraries.zip to s3://test-project-wm/code/libraries.zip
!aws s3 cp /home/glue_user/project_lf/ETL-TDD/libraries.zip s3://test-project-wm/code/dependencies/libraries.zip

# delete everything but the libraries.zip file at s3://test-project-wm/code/
!aws s3 rm s3://test-project-wm/code/batches --recursive

delete: s3://test-project-wm/code/batches/batch2.py
delete: s3://test-project-wm/code/batches/batch1.py
delete: s3://test-project-wm/code/batches/batch3.py
delete: s3://test-project-wm/code/batches/batch4.py
delete: s3://test-project-wm/code/batches/batch5.py


## Upload .py batch files to S3

In [51]:
import boto3

# Create an S3 client
s3_client = boto3.client('s3', region_name='ap-southeast-1')

# Set the bucket and prefix
bucket = 'test-project-wm'
prefix = 'code/batches/'

# Set the list of files to upload
files = [
    '/home/glue_user/project_lf/ETL-TDD/batch1.py',
    '/home/glue_user/project_lf/ETL-TDD/batch2.py',
    '/home/glue_user/project_lf/ETL-TDD/batch3.py',
    '/home/glue_user/project_lf/ETL-TDD/batch4.py',
    '/home/glue_user/project_lf/ETL-TDD/batch5.py',
]

# Upload each file to the S3 bucket
for file_path in files:
    # Get the file name
    file_name = file_path.split('/')[-1]

    # Set the object key
    key = f'{prefix}{file_name}'

    # Upload the file
    s3_client.upload_file(file_path, bucket, key)

print(f'Uploaded {len(files)} files to s3://{bucket}/{prefix}')

Uploaded 5 files to s3://test-project-wm/code/batches/


## Initialise Glue session

In [13]:
# for each script in s3://project-lf/code/jobs/ create a glue job
import boto3

# Create an AWS Glue client
glue_client = boto3.client('glue', region_name='ap-southeat-2')

# Set the parameters for the new Glue job
glue_role = 'data-quality-lf'


## Get a list of all script objects in `project-lf/code/jobs/`

In [14]:
# get all keys in s3://project-lf/code/jobs/
s3_resource = boto3.resource('s3')
bucket = 'test-project-wm'
prefix = 'code/batches/'
objects = s3_resource.Bucket(bucket).objects.filter(Prefix=prefix)
list(objects)

[s3.ObjectSummary(bucket_name='test-project-wm', key='code/batches/batch1.py'),
 s3.ObjectSummary(bucket_name='test-project-wm', key='code/batches/batch2.py'),
 s3.ObjectSummary(bucket_name='test-project-wm', key='code/batches/batch3.py'),
 s3.ObjectSummary(bucket_name='test-project-wm', key='code/batches/batch4.py'),
 s3.ObjectSummary(bucket_name='test-project-wm', key='code/batches/batch5.py')]

## Create a job for every script object

In [16]:
glue_client = boto3.client('glue', region_name='ap-southeast-2')
job_names = []
for s3_object in objects:
    print(s3_object.key)
    
    # Create an AWS Glue client

    # Set the parameters for the new Glue job
    glue_role = 'data-quality-lf'
    script_location = f's3://test-project-wm/{s3_object.key}'
    job_name = script_location.split('/')[-1].split('.')[0]
    extra_py_files = 's3://test-project-wm/code/dependencies/libraries.zip'
    glue_version = '4.0'
    additional_python_modules = "holidays"

    try:
        # Create the new Glue job
        response = glue_client.create_job(
            Name=job_name,
            Role=glue_role,
            Command={'Name': 'glueetl', 'ScriptLocation': script_location},
            GlueVersion=glue_version,
            DefaultArguments={
                '--extra-py-files': extra_py_files,
                '--additional-python-modules': additional_python_modules
            },
            WorkerType='G.1X',
            NumberOfWorkers=2
        )
        print(response)
    except Exception as e:
        print("Skipping job creation for", job_name, e)
    
    job_names.append(job_name)

code/batches/batch1.py
{'Name': 'batch1', 'ResponseMetadata': {'RequestId': '98aca703-2c52-4a92-a51f-3eaadfe4ef46', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 07 Jun 2023 05:59:00 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '17', 'connection': 'keep-alive', 'x-amzn-requestid': '98aca703-2c52-4a92-a51f-3eaadfe4ef46'}, 'RetryAttempts': 0}}
code/batches/batch2.py
{'Name': 'batch2', 'ResponseMetadata': {'RequestId': 'cbb22ab5-7c38-4557-a9d1-638d69331d78', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 07 Jun 2023 05:59:00 GMT', 'content-type': 'application/x-amz-json-1.1', 'content-length': '17', 'connection': 'keep-alive', 'x-amzn-requestid': 'cbb22ab5-7c38-4557-a9d1-638d69331d78'}, 'RetryAttempts': 0}}
code/batches/batch3.py
{'Name': 'batch3', 'ResponseMetadata': {'RequestId': '5474a1bd-1c20-402e-9e0e-df50ec933256', 'HTTPStatusCode': 200, 'HTTPHeaders': {'date': 'Wed, 07 Jun 2023 05:59:00 GMT', 'content-type': 'application/x-amz-json-1.1', 'conte

## Define execution order of jobs

In [17]:
batches = [
     'batch1', 'batch2', 'batch3', 'batch4', 'batch5'
]

## Define a function to run (in parallel) all jobs in a list and wait until they have completed

In [18]:
import boto3
import time
import colorama
from colorama import Fore, Style
colorama.init()

# Create an AWS Glue client
glue_client = boto3.client('glue', region_name='ap-southeast-2')

# Function to run a list of batches and wait for them to finish
def run_batches(batches):
    # Start each job and store its job run ID
    job_run_ids = {}
    for batch_name in batches:
        response = glue_client.start_job_run(JobName=batch_name)
        job_run_id = response['JobRunId']
        job_run_ids[batch_name] = job_run_id
        print(batch_name, job_run_id)
    
    # Wait for all batches to finish
    while True:
        # Check the status of each job run
        all_finished = True
    
        for job_name, job_run_id in job_run_ids.items():
            try:
                response = glue_client.get_job(JobName=job_name)
                response = glue_client.get_job_run(JobName=job_name, RunId=job_run_id)
                status = response['JobRun']['JobRunState']
                if status not in ['SUCCEEDED', 'FAILED', 'STOPPED']:
                    all_finished = False
                else:
                    print(".", end="")
            except glue_client.exceptions.EntityNotFoundException:
                continue
        print("-"*80)
        # If all batches have finished, exit the loop
        if all_finished:
            print("Finished.")
        # Otherwise, wait and check again
        time.sleep(1)

## Execute jobs in order

In [ ]:
run_batches(batches)

## Troubleshooting DFs

In [27]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("test") \
    .getOrCreate()
    


SLF4J: Class path contains multiple SLF4J bindings.
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/spark/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/slf4j-reload4j-1.7.36.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: Found binding in [jar:file:/home/glue_user/aws-glue-libs/jars/log4j-slf4j-impl-2.17.2.jar!/org/slf4j/impl/StaticLoggerBinder.class]
SLF4J: See http://www.slf4j.org/codes.html#multiple_bindings for an explanation.
SLF4J: Actual binding is of type [org.slf4j.impl.Reload4jLoggerFactory]
log4j:WARN No appenders could be found for logger (org.apache.hadoop.metrics2.lib.MutableMetricsFactory).
log4j:WARN Please initialize the log4j system properly.
log4j:WARN See http://logging.apache.org/log4j/1.2/faq.html#noconfig for more info.


### provider_dim

In [49]:
df = spark.read.parquet('s3://test-project-wm/etl/optimised/claim_fact/full/202306072017/')
df.printSchema()

root
 |-- claim_id: integer (nullable = true)
 |-- provider_key: long (nullable = true)
 |-- policyholder_key: long (nullable = true)
 |-- procedure_key: long (nullable = true)
 |-- date_key: integer (nullable = true)
 |-- total_procedure_cost: float (nullable = true)
 |-- medibank_pays: float (nullable = true)
 |-- medicare_pays: float (nullable = true)
 |-- excess: float (nullable = true)
 |-- out_of_pocket: float (nullable = true)



In [45]:
df.show(5)

+------------+--------------------+--------------------+--------+--------------------+----------+-------------------+-------------------+------------------+--------------------+--------------------+
|location_key|             address|              street|postcode|              suburb|track_hash|    record_start_ts|      record_end_ts|record_active_flag|       record_upd_ts|    record_insert_ts|
+------------+--------------------+--------------------+--------+--------------------+----------+-------------------+-------------------+------------------+--------------------+--------------------+
|           0|Apt. 486 646 Mich...|Apt. 486 646 Mich...|    2901| Lake Gabrielchester|      null|1970-01-01 00:00:00|2999-12-31 00:00:00|                 1|2023-06-07 10:08:...|2023-06-07 10:08:...|
|           1|Level 2 85 Rhonda...|Level 2 85 Rhonda...|    2987|           Jimmyfurt|      null|1970-01-01 00:00:00|2999-12-31 00:00:00|                 1|2023-06-07 10:08:...|2023-06-07 10:08:...|
|    

In [40]:
from pyspark.sql.functions import col
df = df.select(
    col("procedure").cast("long"),
    col("procedure_key").cast("long"),
    col("track_hash").cast("long"),
    col("record_start_ts").cast("string"),
    col("record_end_ts").cast("string"),
    col("record_active_flag").cast("short"),
    col("record_upd_ts").cast("string"),
    col("record_insert_ts").cast("string")
)

DataFrame[procedure: bigint, procedure_key: bigint, track_hash: bigint, record_start_ts: string, record_end_ts: string, record_active_flag: smallint, record_upd_ts: string, record_insert_ts: string]

In [47]:
df = spark.read.parquet("s3://test-project-wm/etl/optimised/provider_dim/full/202306072013/")

In [48]:
df.printSchema()

root
 |-- provider_key: long (nullable = true)
 |-- location_key: long (nullable = true)
 |-- provider_license_number: string (nullable = true)
 |-- provider_id: integer (nullable = true)
 |-- provider_name: string (nullable = true)
 |-- provider_phone_number: string (nullable = true)
 |-- provider_email_address: string (nullable = true)
 |-- provider_type: string (nullable = true)
 |-- track_hash: long (nullable = true)
 |-- record_start_ts: string (nullable = true)
 |-- record_end_ts: string (nullable = true)
 |-- record_active_flag: short (nullable = true)
 |-- record_upd_ts: string (nullable = true)
 |-- record_insert_ts: string (nullable = true)



In [33]:
from pyspark.sql.types import StructType, StructField, StringType, LongType, ShortType

schema = StructType([StructField('procedure', LongType(), True), StructField('procedure_key', LongType(), True), StructField('track_hash', LongType(), True), StructField('record_start_ts', StringType(), True), StructField('record_end_ts', StringType(), True), StructField('record_active_flag', ShortType(), True), StructField('record_upd_ts', StringType(), True), StructField('record_insert_ts', StringType(), True)])